In [1]:
spark

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 49, 6, Finished, Available)

# Set up data configuration

In [49]:
blob_account_name = "marckvnonprodblob"
blob_container_name = "bigdata"
# read only
blob_sas_token = "?sv=2021-10-04&st=2023-10-04T01%3A42%3A59Z&se=2024-01-02T02%3A42%3A00Z&sr=c&sp=rlf&sig=w3CH9MbCOpwO7DtHlrahc7AlRPxSZZb8MOgS6TaXLzI%3D"

wasbs_base_url = (
    f"wasbs://{blob_container_name}@{blob_account_name}.blob.core.windows.net/"
)
spark.conf.set(
    f"fs.azure.sas.{blob_container_name}.{blob_account_name}.blob.core.windows.net",
    blob_sas_token,
)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 52, Finished, Available)

# Reading in single parquet file

In [50]:
submissions_path = "reddit-parquet/submissions/"

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 53, Finished, Available)

# select subreddit topics we are going to analyze

In [51]:
topic = ["Tetris","pokemon","SuperMario","GTA","CallOfDuty","FIFA","legostarwars", "assassinscreed","thesims","FinalFantasy"]

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 54, Finished, Available)

# Reading in all of the Reddit data

In [52]:
submissions_df = spark.read.parquet(f"{wasbs_base_url}{submissions_path}")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 55, Finished, Available)

In [53]:
from pyspark.sql.functions import col, asc,desc

submissions_filtered = submissions_df.filter(col("subreddit").isin(topic))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 56, Finished, Available)

# select post with the subreddit topics

In [54]:
df_post = df_save = submissions_filtered.select("subreddit","selftext").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 57, Finished, Available)

In [55]:
df_post_filter = df_post.filter(~col("selftext").isin([" ","[removed]","[deleted]"]))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 58, Finished, Available)

In [56]:
df_post_filter = df_post_filter.dropna()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 59, Finished, Available)

In [73]:
df_post1 = submissions_filtered.select("subreddit","title","selftext").cache()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 70, Finished, Available)

# spark NLP pipeline build

In [2]:
%%configure -f \
{"conf": {"spark.jars.packages": "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.2"}}

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, -1, Finished, Available)

Unrecognized options: 

In [3]:
!pip install spark-nlp

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 6, Finished, Available)

     |████████████████████████████████| 540 kB 9.0 MB/s eta 0:00:01


In [4]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 7, Finished, Available)

In [31]:
documentAssembler = DocumentAssembler()\
    .setInputCol("selftext")\
    .setOutputCol("document")

    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 48, 33, Finished, Available)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


# NLP Cleaning pipeline

In [75]:
df_post2 = df_post1.dropna()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 72, Finished, Available)

In [59]:
documentAssembler = DocumentAssembler() \
    .setInputCol('selftext') \
    .setOutputCol('document')

cleanUpPatterns = ["<[^>]*>"]
# normalizer referred from https://github.com/JohnSnowLabs/spark-nlp/blob/master/examples/python/annotation/text/english/document-normalizer/document_normalizer_notebook.ipynb
documentNormalizer = DocumentNormalizer() \
    .setInputCols("document") \
    .setOutputCol("normalizedDocument") \
    .setAction("clean") \
    .setPatterns(cleanUpPatterns) \
    .setReplacement(" ") \
    .setPolicy("pretty_all") \
    .setLowercase(True)

sentenceDetector = SentenceDetector() \
      .setInputCols(["normalizedDocument"]) \
      .setOutputCol("sentence")
'''
regexTokenizer = Tokenizer() \
      .setInputCols(["sentence"]) \
      .setOutputCol("token") \
      .fit(dataeg)
'''
# tokenization
tokenizer = (
    Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")
)

# make the words back to root form
stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

# remove stop words
stop_words_cleaner = StopWordsCleaner() \
    .setInputCols(["stem"])  \
    .setOutputCol("cleaned_token") \
    .setCaseSensitive(False)

# check spelling
spellModel = ContextSpellCheckerModel\
    .pretrained("spellcheck_dl", "en")\
    .setInputCols("cleaned_token")\
    .setOutputCol("final_token")\

nlpcleanPipeline = \
  Pipeline() \
    .setStages([
        documentAssembler,
        documentNormalizer,
        sentenceDetector,
        tokenizer,
        stemmer,
        stop_words_cleaner,
        spellModel])


StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 62, Finished, Available)

spellcheck_dl download started this may take some time.
Approximate size to download 95.1 MB
[OK!]


In [61]:
cleaned_df = nlpcleanPipeline.fit(df_post_filter).transform(df_post_filter)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 64, Finished, Available)

In [68]:
new_df_post = cleaned_df.select("subreddit", 'final_token.result')

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 71, Finished, Available)

In [69]:
new_df_post.show(10)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 72, Finished, Available)

+------------+--------------------+
|   subreddit|              result|
+------------+--------------------+
|        FIFA|                  []|
|FinalFantasy|                  []|
|     pokemon|[love, outdoor, a...|
|        FIFA|                  []|
|     pokemon|                  []|
|legostarwars|                  []|
|     pokemon|                  []|
|legostarwars|                  []|
|     pokemon|[really, make, wo...|
|     pokemon|[auto=webp&amp, ;...|
+------------+--------------------+
only showing top 10 rows



In [81]:
from pyspark.sql.functions import size, concat_ws

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 84, Finished, Available)

In [74]:
# remove empty array
new_df_post_filter = new_df_post.filter(size(col("result")) > 0)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 77, Finished, Available)

In [75]:
new_df_post_filter.show(10)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 78, Finished, Available)

+---------+--------------------+
|subreddit|              result|
+---------+--------------------+
|  pokemon|[love, outdoor, a...|
|  pokemon|[really, make, wo...|
|  pokemon|[auto=webp&amp, ;...|
|     FIFA|[im, always, 2438...|
|      GTA|[around, 5, min, ...|
|      GTA|[know, Nero, want...|
|  pokemon|[ar, rule, bug, t...|
|  pokemon|[get, 1, :, Chari...|
|  thesims|[2+, teens:must, ...|
|  pokemon|[gamefreak, make,...|
+---------+--------------------+
only showing top 10 rows



In [83]:
# make the array to string
final_df = new_df_post_filter.withColumn("text", concat_ws(", ", col("result")))
final_df_post = final_df.select("subreddit", 'text')
final_df_post.show(10)


StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 86, Finished, Available)

+---------+--------------------+
|subreddit|                text|
+---------+--------------------+
|  pokemon|love, outdoor, ac...|
|  pokemon|really, make, won...|
|  pokemon|auto=webp&amp, ;,...|
|     FIFA|im, always, 2438,...|
|      GTA|around, 5, min, d...|
|      GTA|know, Nero, want,...|
|  pokemon|ar, rule, bug, to...|
|  pokemon|get, 1, :, Charik...|
|  thesims|2+, teens:must, m...|
|  pokemon|gamefreak, make, ...|
+---------+--------------------+
only showing top 10 rows



# Sentiment pipeline

In [84]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 87, Finished, Available)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


In [85]:
senti_model = nlpPipeline.fit(final_df_post)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 88, Finished, Available)

In [76]:
senti_model2 = nlpPipeline.fit(df_post2)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 73, Finished, Available)

In [86]:
senti_pred = senti_model.transform(final_df_post)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 89, Finished, Available)

In [77]:
senti_pred2 = senti_model.transform(df_post2)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 74, Finished, Available)

In [87]:
senti_pred.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 90, Finished, Available)

+---------+--------------------+--------------------+--------------------+--------------------+
|subreddit|                text|            document| sentence_embeddings|           sentiment|
+---------+--------------------+--------------------+--------------------+--------------------+
|  pokemon|love, outdoor, ac...|[{document, 0, 99...|[{sentence_embedd...|[{category, 0, 99...|
|  pokemon|really, make, won...|[{document, 0, 46...|[{sentence_embedd...|[{category, 0, 46...|
|  pokemon|auto=webp&amp, ;,...|[{document, 0, 19...|[{sentence_embedd...|[{category, 0, 19...|
|     FIFA|im, always, 2438,...|[{document, 0, 50...|[{sentence_embedd...|[{category, 0, 50...|
|      GTA|around, 5, min, d...|[{document, 0, 15...|[{sentence_embedd...|[{category, 0, 15...|
+---------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [78]:
senti_pred2.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 75, Finished, Available)

+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|   subreddit|               title|            selftext|            document| sentence_embeddings|           sentiment|
+------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        FIFA|if you every get ...|                    |[{document, 0, -1...|                  []|                  []|
|FinalFantasy|Final Fantasy Pix...|                    |[{document, 0, -1...|                  []|                  []|
|     pokemon|the PokemonTogeth...|So several days a...|[{document, 0, 13...|[{sentence_embedd...|[{category, 0, 13...|
|        FIFA|should i select o...|           [removed]|[{document, 0, 8,...|[{sentence_embedd...|[{category, 0, 8,...|
|        FIFA| Team of the week 17|                    |[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
+------------+--------------------+-----

In [ ]:
from pyspark.sql.functions import col, size

In [79]:
df_filtered = senti_pred2.filter(size(col("sentence_embeddings")) != 0)
df_filtered.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 76, Finished, Available)

+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|subreddit|               title|            selftext|            document| sentence_embeddings|           sentiment|
+---------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  pokemon|the PokemonTogeth...|So several days a...|[{document, 0, 13...|[{sentence_embedd...|[{category, 0, 13...|
|     FIFA|should i select o...|           [removed]|[{document, 0, 8,...|[{sentence_embedd...|[{category, 0, 8,...|
|     FIFA| Team of the week 17|                    |[{document, 0, -1...|[{sentence_embedd...|[{category, 0, 8,...|
|  pokemon|Who's a non-villa...|For me, Tyme insp...|[{document, 0, 66...|[{sentence_embedd...|[{category, 0, 66...|
|  pokemon|Zard Hunting in B...|           [removed]|[{document, 0, 8,...|[{sentence_embedd...|[{category, 0, 8,...|
+---------+--------------------+--------------------+-----------

In [88]:
df_senti = senti_pred.select("subreddit",F.explode('sentiment.result'))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 91, Finished, Available)

In [82]:
df_senti2 = senti_pred2.select("subreddit",F.explode('sentiment.result'))

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 79, Finished, Available)

In [89]:
df_senti = df_senti.withColumnRenamed("col", "sentiment")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 92, Finished, Available)

In [83]:
df_senti2 = df_senti2.withColumnRenamed("col", "sentiment")

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 80, Finished, Available)

In [90]:
df_senti.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 50, 93, Finished, Available)

+---------+---------+
|subreddit|sentiment|
+---------+---------+
|  pokemon|      pos|
|  pokemon|      neg|
|  pokemon|      pos|
|     FIFA|      pos|
|      GTA|      neg|
+---------+---------+
only showing top 5 rows



In [84]:
df_senti2.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 81, Finished, Available)

+---------+---------+
|subreddit|sentiment|
+---------+---------+
|  pokemon|      pos|
|     FIFA|      pos|
|     FIFA|      neg|
|  pokemon|      neg|
|  pokemon|      pos|
+---------+---------+
only showing top 5 rows



In [ ]:
df_senti_count = df_senti.groupBy("subreddit","sentiment").count().toPandas()

In [85]:
df_filtered_use=df_filtered.limit(10000)
df_senti_use=df_senti2.limit(10000)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 82, Finished, Available)

In [87]:
df_filtered_use_final = df_filtered_use.drop(*("document", "sentence_embeddings", "sentiment"))
df_filtered_use_final.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 84, Finished, Available)

+---------+--------------------+--------------------+
|subreddit|               title|            selftext|
+---------+--------------------+--------------------+
|  pokemon|the PokemonTogeth...|So several days a...|
|     FIFA|should i select o...|           [removed]|
|     FIFA| Team of the week 17|                    |
|  pokemon|Who's a non-villa...|For me, Tyme insp...|
|  pokemon|Zard Hunting in B...|           [removed]|
+---------+--------------------+--------------------+
only showing top 5 rows



In [92]:
from pyspark.sql.functions import monotonically_increasing_id

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 89, Finished, Available)

In [93]:
df_senti_text_use = df_filtered_use_final.withColumn("index", monotonically_increasing_id()).join(df_senti_use.withColumn("index", monotonically_increasing_id()),"index").drop("index")
df_senti_text_use.show(5)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 90, Finished, Available)

+---------+--------------------+--------------------+---------+---------+
|subreddit|               title|            selftext|subreddit|sentiment|
+---------+--------------------+--------------------+---------+---------+
|  pokemon|the PokemonTogeth...|So several days a...|  pokemon|      pos|
|     FIFA|should i select o...|           [removed]|     FIFA|      pos|
|     FIFA| Team of the week 17|                    |     FIFA|      neg|
|  pokemon|Who's a non-villa...|For me, Tyme insp...|  pokemon|      neg|
|  pokemon|Zard Hunting in B...|           [removed]|  pokemon|      pos|
+---------+--------------------+--------------------+---------+---------+
only showing top 5 rows



In [111]:
df_senti_text_save=df_senti_text_use.toPandas()

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 108, Finished, Available)

In [41]:
import os
CSV_DIR = os.path.join("Users/ky285/fall-2023-reddit-project-team-11/data", "csv")
df_senti_count.to_csv(f"{CSV_DIR}/sentiment_count_top10.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 13, 34, Finished, Available)

In [112]:
import os
CSV_DIR = os.path.join("Users/zx111/fall-2023-reddit-project-team-11/data", "csv")
df_senti_text_save.to_csv(f"{CSV_DIR}/sentiment_text_top10.csv", index=False)

StatementMeta(3c67b279-1d53-4b7a-b0d9-41cb8b4b6723, 40, 109, Finished, Available)